# Detects Suspicious Commands on Linux systems
Detects relevant commands often related to malware or hacking activity

## Rule Content
```
- title: Detects Suspicious Commands on Linux systems
  id: 1543ae20-cbdf-4ec1-8d12-7664d667a825
  status: experimental
  description: Detects relevant commands often related to malware or hacking activity
  references:
  - Internal Research - mostly derived from exploit code including code in MSF
  date: 2017/12/12
  author: Florian Roth
  logsource:
    product: linux
    service: auditd
    category: null
  detection:
    cmd1:
      type: EXECVE
      a0: chmod
      a1: '777'
    cmd2:
      type: EXECVE
      a0: chmod
      a1: u+s
    cmd3:
      type: EXECVE
      a0: cp
      a1: /bin/ksh
    cmd4:
      type: EXECVE
      a0: cp
      a1: /bin/sh
    condition: 1 of them
  falsepositives:
  - Admin activity
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(type:"EXECVE" AND ((a0:"chmod" AND a1:"777") OR (a0:"chmod" AND a1:"u\+s") OR (a0:"cp" AND a1:"\/bin\/ksh") OR (a0:"cp" AND a1:"\/bin\/sh")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()